# find tracerRNA


In [33]:
# First created Thu Jul 11 09:51:15 CEST 2019
# Made by L-F-S
# At the University Of Trento, Italy

import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna, generic_protein, RNAAlphabet
from Bio import pairwise2


sys.path.insert(0, '/home/lorenzo.signorini/cas_mining/utils/')
import filename_discrepancies
sys.path.insert(0, '/home/lorenzo.signorini/cas_mining/9casminer_v2/')
import locus

feature="Cas9" #WARNING!!! CHANGE THIS!!
#outpath="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/5caslocitable/out/"+feature+"/"
datadir="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/"




In [34]:
# Import data
cas_dataset=pd.read_csv("/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/5caslocitable/known_"+feature+"_variants_table.csv", index_col=0)
cas_dataset.columns

Index(['Seq ID', 'Seq Description', 'Seq', 'Contig', 'Genome Name', 'Study',
       'Sample Name', 'SGB ID', 'Pos', 'pilercr_CRISPR', 'minced_CRISPR',
       'prokka_cas1', 'prokka_cas2', 'uSGB', 'Level of estimated taxonomy',
       'Estimated taxonomy'],
      dtype='object')

In [35]:
seqid ="AOPFDEBE_00006"
print("SEQ ID: ", seqid)
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0])
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0])
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0])
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0])
print(len(cas_dataset[cas_dataset["Seq ID"]==seqid]["Seq"].iloc[0]))
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["minced_CRISPR"].iloc[0])
print("----------------------------------------------------------------------")


SEQ ID:  AOPFDEBE_00006
NODE_411_length_19879_cov_2.8009
Obregon-TitoAJ_2015
Obregon-TitoAJ_2015__SM31__bin.19
17319
978
[('.', '10862', '12283')]
----------------------------------------------------------------------


## check there is only 1 Cas9 in that genome..

In [26]:
genomename=cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0]
cas_dataset[cas_dataset["Genome Name"]==genomename]

,Seq ID,Seq Description,Seq,Contig,Genome Name,Study,Sample Name,SGB ID,Pos,pilercr_CRISPR,minced_CRISPR,prokka_cas1,prokka_cas2,uSGB,Level of estimated taxonomy,Estimated taxonomy
19055,AOPFDEBE_00006,AOPFDEBE_00006 CRISPR-associated endonuclease ...,MSLIFGFDLGIASIGWAAVKMSDPTASDTDAAAPDNRPVGEILGAG...,NODE_411_length_19879_cov_2.8009,Obregon-TitoAJ_2015__SM31__bin.19,Obregon-TitoAJ_2015,SM31,17319,6519 9455,"[('ID=CRISPR1', '11227', '6')]","[('.', '10862', '12283')]","[('ID=AOPFDEBE_00007', '9510', '10424')]","[('ID=AOPFDEBE_00008', '10453', '10758')]",Yes,Other,-


In [39]:
cas9_aa=cas_dataset[cas_dataset["Seq ID"]==seqid]["Seq"].iloc[0]
dataset=cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0]
genomename=cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0]
SGB=cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0]
contigname=cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0]
samplename = genomename.split("__")[1]

print("\n-----------------------------------------------------\n"+feature+" id:\t"+seqid+"\nSGB:\t"+str(SGB)+"\nGenome Name:\t",\
      genomename+"\nContig:\t"+cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0],"\n-----------------------------------------------------\n")

print("..working genomename:", filename_discrepancies.get_originalsamplename_froms3name_of_genome(genomename,dataset))
########################################################################

# get CRISPR spacer and repeat sequence

cr=locus.CRISPRarray(feature=feature, contigname=contigname, genomename=genomename, datasetname=dataset)
cr.get_CRISPR_array()
spacers=cr.spacers
repeats=cr.repeats
repeat_start_pos=cr.repstartpos
unique_repeats=np.unique(repeats)
print("unique repeats:")
for i in range(len(unique_repeats)):
    print("repeat ",str(i+1),":\t",unique_repeats[i])
print("cp "+cr.path+" "+datadir)


# get whole locus sequence todo this could become a function of its own, dentro utils, gli dai seqid della cas e ti dà tt il locus

# > get start and stop position of all Cass. questo devi metterlo nella tabella di cas non ci sono santi.
cas_position={feature:[int(n) for n in cas_dataset[cas_dataset["Seq ID"]==seqid]["Pos"].iloc[0].split()],\
              "Cas1":[int(n) for n in eval(cas_dataset[cas_dataset["Seq ID"]==seqid]["prokka_cas1"].iloc[0])[0][1:]],\
              "Cas2":[int(n) for n in eval(cas_dataset[cas_dataset["Seq ID"]==seqid]["prokka_cas2"].iloc[0])[0][1:]],\
              "CRISPR":[int(n) for n in eval(cas_dataset[cas_dataset["Seq ID"]==seqid]["minced_CRISPR"].iloc[0])[0][1:]]
             }
print("Locus features and positions: ", cas_position)
# so già che all casses inside here are on the same contig which is 'contig', xke le ho estratte per essere così
# FALSO! è cosi in cas_dataset, non in tabellazzza! mi ero confuso! trovato il bug!

genomefilename= "/shares/CIBIO-Storage/CM/scratch/tmp_projects/epasolli_darkmatter/allcontigs/ALLreconstructedgenomes/"+str(SGB)+"/"+genomename+".fa"
print("+"*80)
print(cas_position)
print("Retrieving locus")
print("Copying file ot output...")
print("cp "+genomefilename+" "+datadir)

# access contig:

for record in SeqIO.parse(genomefilename,"fasta"):  #c'è modo di non fare questo 'ciclo'?
    if record.id.startswith(contigname):
        #test se effettivamente la seq di cas è dove voglio io

        SeqIO.write(record, datadir+"tempblastdb", "fasta")
        print("CRISPR si trova", cas_position["CRISPR"])
        print("Cas9 si trova", cas_position[feature])
        print("Cas2 si trova", cas_position["Cas2"])
        print("Cas1 si trova", cas_position["Cas1"])

        #estrai la posizione di cas9 dal contig, e traducila
        temp_alt_cas9start=98503
        temp_alt_cas9stop=101700
        cas9start=cas_position[feature][0]
        cas9stop=cas_position[feature][1]
        cas9_nnseq=record.seq[cas9start-1:cas9stop-3] #gff should have a 1-based positional annotation (ma sto andando a occhio finchè non sono uguali, è già un oggetto Bio.Seq

        my_translated_cas9=cas9_nnseq.transcribe().translate()

  
        print(len(cas9_nnseq)/3,len(cas9_aa))


        if my_translated_cas9==cas9_aa:
            print("Cas locus on plus strand")
            plus=True
        if not my_translated_cas9==cas9_aa:
            #proviamo col reverse complement
            cas9_nnseq=record.seq[cas9start+2:cas9stop]
            my_translated_cas9=cas9_nnseq.reverse_complement().transcribe().translate()
            print("Cas locus on minus strand")
        print("Is Cas9 translated from the genome equal to the orignal annotation?", my_translated_cas9==cas9_aa)
        print(my_translated_cas9)
        print("+"*80)
        print(cas9_aa)
        
        alignments = pairwise2.align.localxx(my_translated_cas9,cas9_aa)
        print(pairwise2.format_alignment(*alignments[0]))
        
        #save .faa aminoacid sequence to output:
        if my_translated_cas9==cas9_aa:
            cas9aafilename=str(SGB)+"__"+seqid+"__"+genomename+"."+feature+".faa"
            f=open(datadir+cas9aafilename,"w") #TODO dir
            f.write(">"+seqid+"\n"+cas9_aa)
            f.close()

        #opzione 1 blast
      #1. salva np.uniq(repeats) in un fasta file: build temporary query file for blastn search

        print("let's now blast the repeat against  the genome")
        os.chdir(datadir)
        tempfile=open("temp_repeat_seq", "w")
        tempfile.close()
        tempfile=open("temp_repeat_seq", "a")

        for n, repeat in enumerate([Seq(sequence) for sequence in np.unique(repeats)]):
            tempfile.write(">rpt"+str(n+1)+"|"+contigname+"|"+genomename+"|"+seqid+"\n"+str(repeat)+"\n") 
        tempfile.close()

        #2. Blast query file against db

        #3. make db file
        print("Building blast database with:")
        dbfile=datadir+"tempblastdb"
        print("makeblastdb -in "+dbfile+" -parse_seqids  -dbtype nucl")
        os.system("makeblastdb -in "+dbfile+" -parse_seqids  -dbtype nucl")


        print("Running blastn of query file agianst all contigs")
        blastoutfile=datadir\
        +str(SGB)+"__"+seqid+"__"+genomename+"__"+feature+".trcr.blastout"
        blastn_command = "blastn -out "+blastoutfile+" -outfmt \"6  qseqid sseqid pident qlen length mismatch gapopen qseq sseq sstart send evalue sstrand\" -query temp_repeat_seq -db "+dbfile+" -evalue 0.001 -word_size 11 -penalty -2"
        print(blastn_command)
        os.system(blastn_command)
       # os.system("rm /shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracerRNA/temp*")



-----------------------------------------------------
Cas9 id:	AOPFDEBE_00006
SGB:	17319
Genome Name:	 Obregon-TitoAJ_2015__SM31__bin.19
Contig:	NODE_411_length_19879_cov_2.8009 
-----------------------------------------------------

..working genomename: ('Obregon-TitoAJ_2015__SM31__bin.19', 'Obregon-TitoAJ_2015')


CRISPR 1   Range: 10862 - 12283

POSITION	REPEAT				SPACER

--------	------------------------------------	------------------------------

10862		ATTATACCAAAGACGATTTGAGAAGCAACCTATAAC	AAGACGCACTCAACAGCACCAATATGCAGG	[ 36, 30 ]

10928		ATTATACCAAAGACGATTTGAGAAGCAACCTATAAC	AACAAGTATACGCAATGTTTAACCAAATCA	[ 36, 30 ]

10994		ATTATACCAAAGACGATTTGAGAAGCAACCTATAAC	AGGTCAGGATTATCAACCACGTATCGATTC	[ 36, 30 ]

11060		ATTATACCAAAGACGATTTGAGAAGCAACCTATAAC	CTATACTAAGGGCATATCAATAAGGACATA	[ 36, 30 ]

11126		ATTATACCAAAGACGATTTGAGAAGCAACCTATAAC	ATGTATATTTCATTGTTGATATTTTCTTTG	[ 36, 30 ]

11192		ATTATACCAAAGACGATTTGAGAAGCAACCTATAAC	CCAGCCATGTTATATATTTGTGCCGCTTGT	[ 36, 30 ]

11258		ATTATACCAAAGA

Running blastn of query file agianst all contigs
blastn -out /shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/17319__AOPFDEBE_00006__Obregon-TitoAJ_2015__SM31__bin.19__Cas9.trcr.blastout -outfmt "6  qseqid sseqid pident qlen length mismatch gapopen qseq sseq sstart send evalue sstrand" -query temp_repeat_seq -db /shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/tempblastdb -evalue 0.001 -word_size 11 -penalty -2


In [28]:
#4. parse blast output: only print output if it is NOT one of the old repeats.
print("\n-----------------------------------------------------\nParsing BLAST output, looking for hit which are NOT the same repeats\ni.e. not starting with:", repeat_start_pos,":\n-----------------------------------------------------\n")
f=open(blastoutfile,"r")
for line in f.readlines():
    start_pos=line.split("\t")[9]
    if start_pos not in repeat_start_pos:
        if str(int(start_pos)+1) not in repeat_start_pos:
            if str(int(start_pos)-1) not in repeat_start_pos:
                print(line) 


-----------------------------------------------------
Parsing BLAST output, looking for hit which are NOT the same repeats
i.e. not starting with: ['10862', '10928', '10994', '11060', '11126', '11192', '11258', '11324', '11390', '11456', '11522', '11588', '11654', '11720', '11786', '11852', '11918', '11984', '12050', '12116', '12182', '12248'] :
-----------------------------------------------------

rpt2|NODE_411_length_19879_cov_2.8009|Obregon-TitoAJ_2015__SM31__bin.19|AOPFDEBE_00006	NODE_411_length_19879_cov_2.8009	97.14	36	35	1	0	ATTATACCAAAGACGATTTGAGAAGCAACCTATAA	ATTATATCAAAGACGATTTGAGAAGCAACCTATAA	14317	14283	4e-13	minus

rpt2|NODE_411_length_19879_cov_2.8009|Obregon-TitoAJ_2015__SM31__bin.19|AOPFDEBE_00006	NODE_411_length_19879_cov_2.8009	100.00	36	19	0	0	GACGATTTGAGAAGCAACC	GACGATTTGAGAAGCAACC	6376	6358	6e-06	minus



## extract sequences

In [36]:
# manual input
# COSA NON SO A PRIORI: l'orientamento del CRISPRarray (e quindi dell'antirepeat)
# COSA SO A PRIORI: una cosa sullo strand + si legge SEMPRE E SOLO da SINISTRA VERSO DESTRA
#     una cosa sullo strand - si legge SEMPRE E SOLO da DESTRA VERSO SINISTRA
# il DNA, come molecola, è polarizzato. LE basi sono come un parcheggio a spina di pesce,
# con un angolo e
# quindi una direzione.
# COSA DEVO TROVARE: il match della repeat, e un terminatore nelle sue vicinanze. 
# SE è alla sua DESTRA
# devo prendere il filamento sullo strand + 
# (che potrebbe essere SIA la repeat che ha matchato il BLAST
# direttamente SIA il suo reverse complement. NOn lo so prima xke nn so dove stia
# (== su che strand e in che direzuione sia scritto) CRISPR, lo
# indurrò a posteriori)
# ; SE è  a SINISTRA,
# devo prendere il filamento sullo strand -.
#
# devo, dunque, SCEGLIERE se l'antirepeat è la matched repeat o il suo rev complement 
# a seconda di DOVE STA IL TERMINATORE
# così facendo, identificherò anche l'orinetamento del CRISPR
#
#ESEMPIO
#  se .la matched repeat  è sullos trand - VUOL DIRE CHE VA LETTA DA DESTRA VERSO SINISTRA
# e bisogna vedere dove sara il terminator:
# se è A DESTRA della matched e sullo strnaad + (== si legge da sinistra)
# , allora l'antirepeat sarà effettivamente il reverse complement della matched repeat
#, se  è  A SINISTRA della matched e sullo strand -(==si legge da destra), allora
# il mio antirepeat è in effetti la matched_repeat, e quindi probabilmente il CRISPRarray( essendo palindromo)
# è stato annotato sullo strand opposto. 
# Se il terminatore è A DESTRA della matched (sempre nel caso matched  su -)
# e sullo strand - allora è un terminatore SBAGLIATO
#
mstrand=-1  #manual input: strand of matched repeat
matched_rep_start=6358  #manual input ALWAYS input the smaller number here, even if ti matches on - strand
matched_rep_stop=6376   #manual input



In [44]:
# this is the sequence matching the repeat:
matched_rep=Seq("GACGATTTGAGAAGCAACC")  #manual input
# siccome non mi interessa per ora sapere il verso del CRISPRarray, facciamo che la matchedrepeat e il su revocm
# li chiamo matched_rpt_fwd e matched_rpt_rev:

matched_rpt_fwd=matched_rep if mstrand == 1 else  matched_rep.reverse_complement()
matched_rpt_rev=matched_rep if mstrand== -1 else matched_rep.reverse_complement()

#rpts which match:
rpt=Seq(unique_repeats[1])  #manual input
initial_repeat=rpt

#this should go frombo to bo  = x nt  (x+1)
print("matched rep fwd:", matched_rpt_fwd,"matched rep rev: ", matched_rpt_rev)
len(matched_rpt_fwd)



matched rep fwd: GGTTGCTTCTCAAATCGTC matched rep rev:  GACGATTTGAGAAGCAACC


19

In [38]:
# print terminators

f=open(datadir+str(SGB)+"__"+seqid+"__"+genomename+".terminators")
for line in f.readlines():
    if line.strip().startswith(""): #manual input
        print(line)
f.close()

>NODE_411_length_19879_cov_2.8009

    493  Rnamotif - ATAACAAAAAATGCCCGCCAGTTGACGGGCAcaTTTTATCGACAG -10.20

    496      Both + TCGATAAAATGTGCCCGTCAACTGGCGGGCATTTTTTGTTATA -14.50

    541      Both - CACCAGAATTACCGCGGGAAACCGCGGTTTTTTGTTTGT -12.80

   3277      Both - ATTTAGTACAAAACAAAAATCCCGCATTACGCGGGATTCAGTCCAATATTTGTTaTTTTTTATGTTTA NA

   3298      Both + ATTGGACTGAATCCCGCGTAATGCGGGATTTTTGTTTTGT -10.40

   5407      Both - TCGCGTTTGCGTCGCTGTTCGCAGCGGTTGTTTTTTTCG -7.10

   6226      Both - CGACATAAAAACACCCGCAGGCGGGTGTTTTGTTTTATC -9.90

   6227      Both + ATAAAACAAAACACCCGCCTGCGGGTGTTTTTATGTCGT -9.90

   6228  Rnamotif - ACATAAAAACACCCGCAGGCGGGTGTTTTGTTTTA -5.60

   6231      Both + AACAAAACACCCGCCTGCGGGTGTTTTTATGTCGT -5.40

   6269      Both - ATCCCCTGAAACCGTCCCCCTGCGGGGACGGTTTCTGTTGTATT -16.80

   6271     Erpin + TACAACAGAAACCGTCCCCGCAGGGGGACGGTTTCAGGGGATAC -18.10

  10815     Erpin - AGACACATAAAAACCCGCTGAAATTTGCGGGTTtTTATGTGTTTGTA NA

  10818      Both + AAACACATAAAAACCCGCAAATTT

In [49]:
#29/01/2020
def get_real_repeat(rpt, crisprstrand):
    return rpt if mstrand == 1 else rpt.reverse_complement()

In [68]:
#29/01/2020 final fix
tstrand=-1  #manual input, terminator strand. TODO CHECK che se lo strand è -1 allora è <matched rep (tracrRNA on - strand),e viceversa

terminator_start=6269 #manual input N.b. this is always the smallest position (between start and end)
terminator=Seq("ATCCCCTGAAACCGTCCCCCTGCGGGGACGGTTTCTGTTGTATT".upper()) #manual input
terminator_end=terminator_start+len(terminator)
if tstrand==1:  # if terminator on + strand
    if matched_rep_stop<terminator_start: # check that term is on the RIGHT SIDE of the repeat
        print("Strand OK")
        antirepeat=matched_rpt_fwd
        tracrstrand=tstrand
        print("antirepeat: ",antirepeat)
        crisprstrand =  -mstrand #-mstrand if mstrand == tstrand else -mstrand
        rpt=get_real_repeat(rpt, crisprstrand)
    else:
        print("STRAND NOT OK", matched_rep_start, matched_rep_stop)
elif tstrand==-1:
    if matched_rep_start>terminator_start: # check that term is on the LEFT SIDE of the repeat
        print("Strand OK")
        antirepeat=matched_rpt_rev
        tracrstrand=tstrand
        #todo rpt= ?
        print("antirepeat: ",antirepeat)
        crisprstrand = mstrand # cioé: mstrand if mstrand == tstrand else mstrand
        rpt=get_real_repeat(rpt, crisprstrand)

    else:
        print("STRAND NOT OK", matched_rep_start, matched_rep_stop)
else:
    print("input t strand please")


print("terminator length: ", len(terminator))
print("terminator start and stop:",terminator_start, terminator_end)
#print("nb start and stop are just the position from smallest to biggest, without direction. Pay attention when working on minus strand")
print("tracr strand=",tracrstrand)
print("CRISPR strand=", crisprstrand)
print("Repeat:", rpt,"\t(original repeat:\t", initial_repeat+")")

Strand OK
antirepeat:  GACGATTTGAGAAGCAACC
terminator length:  44
terminator start and stop: 6269 6313
tracr strand= -1
CRISPR strand= -1
Repeat: GTTATAGGTTGCTTCTCAAATCGTCTTTGGTATAAT 	(original repeat:	 ATTATACCAAAGACGATTTGAGAAGCAACCTATAAC)


In [81]:
# Open genome and extract the whole sequence
# find this inside the genome, and link it to the thing found by ARNold:

#I need to specify BOTH the strand and the direction
#if strand is +, then the actual antirepeat is going to be what i called antirepeat 
#only if the repeat blast matched on the minus strand (vedi schema), e vari ncastri complciati"

for record in SeqIO.parse(genomefilename, "fasta"):
    if record.id==contigname:
        contigseq=record.seq

#check antirepeat
extracted_antirepeat =  contigseq[matched_rep_start-1:matched_rep_stop] if tracrstrand==1 else contigseq[matched_rep_start-1:matched_rep_stop].reverse_complement()
print("test antirepeat: ", antirepeat, len(antirepeat), extracted_antirepeat==antirepeat) # -1 why? Indexing diverso evidentemente col blastout, vbb

# check terminator
extracted_terminator =  contigseq[terminator_start-1:terminator_end-1] if tracrstrand==1 else contigseq[terminator_start-1:terminator_end-1].reverse_complement()
print("test terminator: " ,terminator, len(terminator), extracted_terminator==terminator)


pre_tracrRNA= contigseq[matched_rep_start:terminator_end] if tracrstrand==1 else contigseq[terminator_start:matched_rep_stop].reverse_complement()
print("pre tracrRNA:", pre_tracrRNA, len(pre_tracrRNA))

# add mario
mario= contigseq[matched_rep_stop:terminator_start-1] if tracrstrand==1 else contigseq[terminator_end:matched_rep_start-1].reverse_complement()
print("mario sequence:",  mario, len(mario))  # ho fatto i check, il -1 sopra è giusto

test antirepeat:  GACGATTTGAGAAGCAACC 19 True
test terminator:  ATCCCCTGAAACCGTCCCCCTGCGGGGACGGTTTCTGTTGTATT 44 True
pre tracrRNA: GACGATTTGAGAAGCAACCGTTTATATAATAAAGAATATTTTCTGTGGGGTTCTACGATTATGTATCCCCTGAAACCGTCCCCCTGCGGGGACGGTTTCTGTTGTAT 107
mario sequence: GTTTATATAATAAAGAATATTTTCTGTGGGGTTCTACGATTATG 44


In [82]:
print(terminator_end, matched_rep_stop)

6313 6376


In [84]:
# Now, align with the rest of the stuff to see what's up
# first up, with the repeat, lets take the rpt3, which has only got 1 mismatch
# x AOPFDEBE: extracted antirepeat: matched_stop+11

alignments = pairwise2.align.localxx(rpt, antirepeat.reverse_complement())  #29/01/2020 fixed
print(pairwise2.format_alignment(*alignments[0]))
print("rpt length:", len(rpt))


GTTATAGGTTGCTTCTCAAATCGTCTTTGGTATAAT
      |||||||||||||||||||
------GGTTGCTTCTCAAATCGTC-----------
  Score=19

rpt length: 36


In [67]:
# align antirepat + mario with repeat
alignments = pairwise2.align.globalxx( rpt,(antirepeat+mario[:4]).reverse_complement())
print(pairwise2.format_alignment(*alignments[0]))

ATTATACCAAAGACGATTTGAGAAGCAAC--CT-ATAA-C---
|  | | |   |  |  ||     |   |  || | || |   
A--A-A-C---G--G--TT-----G---CTTCTCA-AATCGTC
  Score=16



In [125]:
#no non fa nulla quel CTTT in più

In [126]:
# I would probably say that the tracr is just, simply la parte di pre_tracrRNA la cui antirepeat si lega alla repeat, 
# che, in questo caso, è tutta.
tracrRNA = pre_tracrRNA

print("\n----------------------------------------------------------------------")
print("         tracrRNA sequence for", feature,"protein id", seqid)
print("----------------------------------------------------------------------\n")
print("- contig name:", cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0])
print("- Dataset: ",cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0])
print("- Bin: ",cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0])
print("- SGB: ",cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0])
print("----------------------------------------------------------------------\n")
print("- Length: ", len(tracrRNA),"\n- position: ", terminator_end,matched_rep_stop)
print(tracrRNA.transcribe())
print(" - chosen repeat:\n", rpt) #TODO manca l'orientamento finale


----------------------------------------------------------------------
         tracrRNA sequence for Cas9 protein id FBDDNBEO_00645
----------------------------------------------------------------------

- contig name: NODE_2092_length_11346_cov_6.0000_ID_47560
- Dataset:  ZeeviD_2015
- Bin:  ZeeviD_2015__PNP_Main_471__bin.13
- SGB:  8770
----------------------------------------------------------------------

- Length:  87 
- position:  384 318
GAAAAUUUUUGUUGUAAAGUCUUAACUGAUUUUGCCGAAUUUUAAGCUCUGCAUGGCACCUUGGAAUUCGGCAUUUUUUUUCAAAAC
